In [1]:
#uses Registered2to1Adder_NIR - v1.1
#registerings implemented based on the assumption that delay of Registered2to1Adder_NIR - v1.1 is 1 clk

In [2]:
import sys
import math
import io

In [3]:
#N>=2
N = input('Enter N: ')

Enter N: 8


In [4]:
#IN_WIDTH>=1
IN_WIDTH = input('Enter IN_WIDTH default value (empty for 10): ')

Enter IN_WIDTH default value (empty for 10): 36


In [5]:
N = int(N)
if N<2:
    N=2
if not IN_WIDTH:
    IN_WIDTH=10
else:
    IN_WIDTH = int(IN_WIDTH)
    if IN_WIDTH<1:
        IN_WIDTH=1

In [6]:
__Print_To_File = 0

if __Print_To_File<=0:
    of=sys.stdout
else:
    of=open("./AdderTree_{}.v".format(N), 'w+')

In [7]:
print("`timescale 1ns / 1ps\n", file=of)
print("module AdderTree_{}".format(N), file=of)
print("#(", file=of)
print("parameter IN_WIDTH = {}".format(IN_WIDTH), file=of)
print(")(", file=of)
print("input clk, reset, enable,", file=of)
print("input inReady,", file=of)
print("input signed [IN_WIDTH-1:0] ", end='', file=of)
for i in range(N):
    print("A{}, ".format(i), end='', file=of)
print(file=of)
print("output outReady,", file=of)
lgn=math.ceil(math.log(N,2))
al=-1+lgn
if al==0:
    print("output signed [IN_WIDTH:0] Out,", file=of)
else:
    print("output signed [IN_WIDTH+{}:0] Out,".format(al), file=of)
print("output earlyOutReady", file=of)
print(");\n", file=of)

`timescale 1ns / 1ps

module AdderTree_8
#(
parameter IN_WIDTH = 36
)(
input clk, reset, enable,
input inReady,
input signed [IN_WIDTH-1:0] A0, A1, A2, A3, A4, A5, A6, A7, 
output outReady,
output signed [IN_WIDTH+2:0] Out,
output earlyOutReady
);



In [8]:
regsBuf=io.StringIO()
AIBuf=io.StringIO()

lgN=math.ceil(math.log2(N))
maxLevel=lgN+1
sizeA=[]
for j in range(N):
    sizeA.append(0)
i=0
level=1
maxn=N-1
new=maxn
ORimp=False
REGimp=False
while 1 :
    if level==maxLevel :
        break
    if i<maxn :
        new=new+1
        sizeA.append(sizeA[i]+1)
        if sizeA[i]==0:
            AIBuf.write("wire signed [IN_WIDTH:0] M{};\n".format(new))
            AIBuf.write("Registered2to1Adder_NIR #(.IN_WIDTH(IN_WIDTH))\nAdd{}(\nclk, reset, enable, \n".format(new))
            AIBuf.write("inReady,\nA{}, A{},\n".format(i, i+1))
        else:
            AIBuf.write("wire signed [IN_WIDTH+{}:0] M{};\n".format(sizeA[i], new))
            esize=sizeA[i]-sizeA[i+1]
            if(esize!=0):
                if sizeA[i]==1:
                     AIBuf.write("wire signed [IN_WIDTH:0] M{}e=".format(i+1))
                else:
                     AIBuf.write("wire signed [IN_WIDTH+{}:0] M{}e=".format(sizeA[i]-1, i+1))
                AIBuf.write("{{")
                AIBuf.write("{}".format(esize))
                AIBuf.write("{")
                AIBuf.write("M{}[IN_WIDTH".format(i+1))
                if sizeA[i+1]==0:
                    AIBuf.write("-1")
                elif sizeA[i+1]>1:
                    AIBuf.write("+{}".format(sizeA[i+1]-1))
                AIBuf.write("]}},")
                AIBuf.write("M{}".format(i+1))
                AIBuf.write("};\n")
            AIBuf.write("Registered2to1Adder_NIR #(.IN_WIDTH(IN_WIDTH+{}))\nAdd{}(\nclk, reset, enable, \n".format(sizeA[i], new))
            AIBuf.write("OR[{}],\nM{}, M{}".format(level-2, i, i+1))
            if(sizeA[i+1]!=sizeA[i]):
                AIBuf.write("e");
            AIBuf.write(",\n");
        if not ORimp:
            AIBuf.write("OR[{}],\n".format(level-1))
            ORimp=True
        else:
            AIBuf.write("ORa{}, //not used\n".format(new))
        AIBuf.write("M{},\n".format(new))
        if level==maxLevel-1:
            AIBuf.write("earlyOutReady);\n\n".format(new, new))        
        else :
            AIBuf.write("aeor{}); //not used\n\n".format(new, new))
        if (i+1)==maxn :
            maxn=new
            level=level+1
            ORimp=False
        i=i+2
    else:
        new=new+1
        sizeA.append(sizeA[i])
        if sizeA[new]==0:
            print("reg signed [IN_WIDTH-1:0] M{};".format(new), file=of)
        elif sizeA[new]==1:
            print("reg signed [IN_WIDTH:0] M{};".format(new), file=of)
        else:
            print("reg signed [IN_WIDTH+{}:0] M{};".format(sizeA[new]-1, new), file=of)
        if level==1:
            regsBuf.write("\t\tif(inReady) begin\n")
            regsBuf.write("\t\t\tM{} <= A{};\n".format(new, i))
            regsBuf.write("\t\tend\n")
        else:
            regsBuf.write("\t\tif(OR[{}]) begin\n".format(level-2))
            regsBuf.write("\t\t\tM{} <= M{};\n".format(new, i))      
            regsBuf.write("\t\tend\n")
        i=i+1
        maxn=new
        level=level+1
        ORimp=False
        REGimp=True

In [9]:
print("wire [0:{}] OR;\n".format(lgN-1), file=of)
if REGimp:
    print("always @(posedge clk) begin", file=of)
    print("\tif(reset) begin", file=of)
    print("\t\t//do nothing", file=of)
    print("\tend", file=of)
    print("\telse if(enable) begin", file=of)
    print(regsBuf.getvalue(), end='',file=of)
    print("\tend", file=of)
    print("end\n", file=of)

wire [0:2] OR;



In [10]:
print(AIBuf.getvalue(), end='',file=of)
print("assign Out = M{};".format(new), file=of)
print("assign outReady = OR[{}];".format(lgN-1), file=of)

wire signed [IN_WIDTH:0] M8;
Registered2to1Adder_NIR #(.IN_WIDTH(IN_WIDTH))
Add8(
clk, reset, enable, 
inReady,
A0, A1,
OR[0],
M8,
aeor8); //not used

wire signed [IN_WIDTH:0] M9;
Registered2to1Adder_NIR #(.IN_WIDTH(IN_WIDTH))
Add9(
clk, reset, enable, 
inReady,
A2, A3,
ORa9, //not used
M9,
aeor9); //not used

wire signed [IN_WIDTH:0] M10;
Registered2to1Adder_NIR #(.IN_WIDTH(IN_WIDTH))
Add10(
clk, reset, enable, 
inReady,
A4, A5,
ORa10, //not used
M10,
aeor10); //not used

wire signed [IN_WIDTH:0] M11;
Registered2to1Adder_NIR #(.IN_WIDTH(IN_WIDTH))
Add11(
clk, reset, enable, 
inReady,
A6, A7,
ORa11, //not used
M11,
aeor11); //not used

wire signed [IN_WIDTH+1:0] M12;
Registered2to1Adder_NIR #(.IN_WIDTH(IN_WIDTH+1))
Add12(
clk, reset, enable, 
OR[0],
M8, M9,
OR[1],
M12,
aeor12); //not used

wire signed [IN_WIDTH+1:0] M13;
Registered2to1Adder_NIR #(.IN_WIDTH(IN_WIDTH+1))
Add13(
clk, reset, enable, 
OR[0],
M10, M11,
ORa13, //not used
M13,
aeor13); //not used

wire signed [IN_WIDTH+2:0] M1

In [11]:
print("endmodule", file=of)

endmodule


In [12]:
if __Print_To_File>0:
    of.close()